<a href="https://colab.research.google.com/github/nntu079/Basic-Pytorch/blob/main/BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#Nx1x28x28 
# Hyperparameters
input_size=28
sequence_length=28
num_layers=2
hidden_size=256
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=2

In [4]:
#Create Bidirectional BRNN
class BRNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super(BRNN,self).__init__()
    self.hidden_size=hidden_size
    self.num_layers=num_layers
    self.LSTM=nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,bidirectional=True)
    self.fc = nn.Linear(hidden_size*2,num_classes)
  def forward(self,x):
    h0=torch.zeros(self.num_layers*2,x.size(0),self.hidden_size).to(device)
    c0=torch.zeros(self.num_layers*2,x.size(0),self.hidden_size).to(device)

    #_ là (h,c) 
    out,_=self.LSTM(x,(h0,c0))
    out=self.fc(out[:,-1,:])

    return out

model = BRNN(input_size,hidden_size,num_layers,num_classes);
x=torch.randn(batch_size,sequence_length,input_size);

print(x.shape)
print(model(x).shape)

torch.Size([64, 28, 28])
torch.Size([64, 10])


In [5]:
# Load Data
patch=r'/content/drive/MyDrive/Môn học/Pytorch/mnist';
train_dataset=datasets.MNIST(root=patch,train=True,transform=transforms.ToTensor(),download=True)
train_loader= DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset=datasets.MNIST(root=patch,train=False,transform=transforms.ToTensor(),download=True)
test_loader= DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [6]:
# Initialize Network
model= BRNN(input_size,hidden_size,num_layers,num_classes).to(device)

In [7]:
# Loss and optimizer
criterion= nn.CrossEntropyLoss();
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [8]:
x=torch.randn(batch_size,sequence_length,input_size);
print(x.shape)
x=x.reshape(1,64,28,28)

x=x.squeeze(1)
print(x.shape)

torch.Size([64, 28, 28])
torch.Size([1, 64, 28, 28])


In [12]:
# Train Network
for epoch in range(num_epochs):
    print(epoch)
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        print(batch_idx,loss)
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

0
0 tensor(0.0152, grad_fn=<NllLossBackward>)
1 tensor(0.1660, grad_fn=<NllLossBackward>)
2 tensor(0.0976, grad_fn=<NllLossBackward>)
3 tensor(0.1659, grad_fn=<NllLossBackward>)
4 tensor(0.0889, grad_fn=<NllLossBackward>)
5 tensor(0.1403, grad_fn=<NllLossBackward>)
6 tensor(0.0686, grad_fn=<NllLossBackward>)
7 tensor(0.0544, grad_fn=<NllLossBackward>)
8 tensor(0.0945, grad_fn=<NllLossBackward>)
9 tensor(0.0568, grad_fn=<NllLossBackward>)
10 tensor(0.1729, grad_fn=<NllLossBackward>)
11 tensor(0.0810, grad_fn=<NllLossBackward>)
12 tensor(0.0274, grad_fn=<NllLossBackward>)
13 tensor(0.0129, grad_fn=<NllLossBackward>)
14 tensor(0.1627, grad_fn=<NllLossBackward>)
15 tensor(0.1129, grad_fn=<NllLossBackward>)
16 tensor(0.0118, grad_fn=<NllLossBackward>)
17 tensor(0.2402, grad_fn=<NllLossBackward>)
18 tensor(0.1515, grad_fn=<NllLossBackward>)
19 tensor(0.0117, grad_fn=<NllLossBackward>)
20 tensor(0.1698, grad_fn=<NllLossBackward>)
21 tensor(0.2167, grad_fn=<NllLossBackward>)
22 tensor(0.1336, 

In [13]:
# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with \
              accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )
    # Set model back to train
    model.train()
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59372 / 60000 with               accuracy 98.95
Checking accuracy on test data
Got 9855 / 10000 with               accuracy 98.55
